In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/texts/"

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SimpleRNN, GRU
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import os

books = [path + '6.txt']
def text_from_file(filepath):
    text = ""
    with open(filepath, 'rb') as book:
        text = book.read()#.decode(encoding='utf-8')
    return text

raw_text = ''
for book in books:
    raw_text += text_from_file(book)#[766:-150]

#raw_text = raw_text.lower()

vocabulary = sorted(list(set(raw_text)))
char2idx = {u: i for i, u in enumerate(vocabulary)}
idx2char = np.array(vocabulary)
text_to_int = np.array([char2idx[c] for c in raw_text])
n_chars = len(raw_text)
n_vocab = len(vocabulary)
print('Всего символов: ', n_chars)
print('Всего слов в словаре: ', n_vocab)
seq_length = 100
examples_per_epoch = len(raw_text) // (seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_to_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(part):
    input_text = part[:-1]
    target_text = part[1:]
    return input_text, target_text

batch_size = 32
num_epochs = 30
data = sequences.map(split_input_target)
data = data.shuffle(10000).batch(batch_size, drop_remainder=True)

Всего символов:  426016
Всего слов в словаре:  136


In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
lstm_sing = Sequential([
    Embedding(n_vocab, 256,
              batch_input_shape=[batch_size, None]),
    LSTM(1024,
         return_sequences=True,
         stateful=True,
         recurrent_initializer='glorot_uniform'),
    Dense(n_vocab)
])
lstm_sing.summary()
lstm_sing.compile(loss=loss, optimizer='adam')
lstm_one = lstm_sing.fit(data, epochs=num_epochs)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (32, None, 256)           34816     
                                                                 
 lstm_5 (LSTM)               (32, None, 1024)          5246976   
                                                                 
 dense_5 (Dense)             (32, None, 136)           139400    
                                                                 
Total params: 5,421,192
Trainable params: 5,421,192
Non-trainable params: 0
_________________________________________________________________
Epoch 1/60
131/131 [==============================] - 12s 71ms/step - loss: 2.9190
Epoch 2/60
131/131 [==============================] - 7s 50ms/step - loss: 2.3526
Epoch 3/60
131/131 [==============================] - 6s 43ms/step - loss: 2.0826
Epoch 4/60
131/131 [==============================] - 7s 50ms

In [ ]:
def generate_text(model, start_string, num_generate, temperature=1):
    input_eval = np.array([char2idx[s] for s in start_string])
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return start_string + ''.join(text_generated)

In [ ]:
lstm_sing.build(tf.TensorShape([1, None]))
generate_text(lstm_sing, 'привееееееееет, меня зовут костя', 50)

"привееееееееет, меня зовут костяhjSIШ§rb*xNp1xer[+JAe[iNx'ШЬXJХsiEЦl1EihXpШb§'[WoM"

In [ ]:
lstm_double =  Sequential([
                         Embedding(n_vocab, 256,
                                   batch_input_shape=[batch_size, None]),
                         LSTM(1024,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer='glorot_uniform'),
                         LSTM(1024,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer='glorot_uniform'),
                         Dense(n_vocab)
                         ])
lstm_double.summary()



lstm_double.compile(loss= loss, optimizer='adam')


lstm_double_dir = './lstm2_checkpoints'
lstm_double_prefix = os.path.join(lstm_double_dir, "checkpoint_{epoch}")
lstm_double_checkpoint = ModelCheckpoint(filepath=lstm_double_prefix, save_weights_only=True)


lstm_two = lstm_double.fit(data, epochs=num_epochs,callbacks=[lstm_double_checkpoint])

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (32, None, 256)           34816     
                                                                 
 lstm_9 (LSTM)               (32, None, 1024)          5246976   
                                                                 
 lstm_10 (LSTM)              (32, None, 1024)          8392704   
                                                                 
 dense_9 (Dense)             (32, None, 136)           139400    
                                                                 
Total params: 13,813,896
Trainable params: 13,813,896
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
131/131 [==============================] - 24s 142ms/step - loss: 3.0646
Epoch 2/20
131/131 [==============================] - 16s 111ms/step - loss: 2.3730
Epoch 

In [ ]:
lstm_double =  Sequential([
                         Embedding(n_vocab, 256,
                                   batch_input_shape=[1, None]),
                         LSTM(1024,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer='glorot_uniform'),
                         LSTM(1024,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer='glorot_uniform'),
                         Dense(n_vocab)
                         ])



lstm_double.load_weights(tf.train.latest_checkpoint(lstm_double_dir))



lstm_double.build(tf.TensorShape([1, None]))
generate_text(lstm_double, 'привет, меня зовут костя', 50)

'привет, меня зовут костянуть с учительница у меня и Эль\r\nразрие есть  кото'

In [ ]:
rnn = Sequential([
                  Embedding(n_vocab, 256,
                            batch_input_shape=[batch_size, None]),
                  SimpleRNN(1024,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                  Dense(n_vocab)
                  ])

rnn.summary()



rnn.compile(loss= loss, optimizer='adam')


rnn_dir = './rnn_checkpoints'
rnn_prefix = os.path.join(rnn_dir, "checkpoint_{epoch}")
rnn_checkpoint = ModelCheckpoint(filepath=rnn_prefix, save_weights_only=True)


RNN = rnn.fit(data, epochs=num_epochs,callbacks=[rnn_checkpoint])

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (32, None, 256)           34816     
                                                                 
 simple_rnn (SimpleRNN)      (32, None, 1024)          1311744   
                                                                 
 dense_11 (Dense)            (32, None, 136)           139400    
                                                                 
Total params: 1,485,960
Trainable params: 1,485,960
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
131/131 [==============================] - 17s 115ms/step - loss: 2.9185
Epoch 2/20
131/131 [==============================] - 13s 91ms/step - loss: 2.3928
Epoch 3/20
131/131 [==============================] - 15s 107ms/step - loss: 2.1951
Epoch 4/20
131/131 [==============================] - 13

In [ ]:


rnn = Sequential([
                  Embedding(n_vocab, 256,
                            batch_input_shape=[1, None]),
                  SimpleRNN(1024,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                  Dense(n_vocab)
                  ])



rnn.load_weights(tf.train.latest_checkpoint(rnn_dir))



rnn.build(tf.TensorShape([1, None]))
generate_text(rnn, 'привет, меня зовут костя', 50)

'привет, меня зовут костяM\nLETSЬ*******yk/\nGiGЪЬgЪkЬ******ncw********kSb***'